In [1]:
from qcutils.credential import load_provider
from qcutils.benchmarks import QASMBenchmark
from qcutils.pulse import CircAmpTimeSeries, BasisPulse
from qcutils.pulse import CircAmpTimeSeries
from qcutils.pulse.amptimeseries.metric import dist, norm

from qiskit import transpile, schedule


provider = load_provider()
backend_list = provider.backends(simulator=False)

bm = QASMBenchmark(r"../../QASMBench", "small", num_qubits_list=list(range(8)), remove_final_measurements=True, do_transpile=False)

In [4]:
# save the current basis pulse library

for backend in backend_list:
    bp = BasisPulse(backend)
    bp.save_basis_amp_time_series_list(f"./data/basis_pulse_{backend.configuration().backend_name}.pickle")

In [5]:
def min_normalized_circ_dist_in_backend_list(circ, backend_list, **trasnpile_args):
    import numpy as np
    min_normalized_circ_dist = float("inf")
    circ_total_list = [CircAmpTimeSeries.from_circ(transpile(circ, b, **trasnpile_args), b).total() for b in backend_list]
    for i, circ_total in enumerate(circ_total_list[:-1]):
        dist_list = [dist(circ_total, total_b) for total_b in circ_total_list[i+1:]]
        min_dist_idx = np.argmin(dist_list)
        curr_min = dist_list[min_dist_idx]
        normalized_min_a = curr_min / norm(circ_total)
        normalized_min_b = curr_min / norm(circ_total_list[min_dist_idx])
        if normalized_min_a < min_normalized_circ_dist or normalized_min_b < min_normalized_circ_dist:
            if normalized_min_a < normalized_min_b:
                min_idx_pair = (i, i + 1 + min_dist_idx)
                min_normalized_circ_dist = normalized_min_a
            else:
                min_idx_pair = (i + 1 + min_dist_idx, i)
                min_normalized_circ_dist = normalized_min_b
    return min_normalized_circ_dist, min_idx_pair

In [6]:
def result(bm, provider, **transpile_args):
    print("Benchmark\t\tMin CD (QP)\tMin IDX Pair")

    for circ_name in bm.circ_name_list:
        num_qubits = bm.num_qubits(circ_name)
        backend_list = provider.backends(simulator=False, filters=lambda b: b.configuration().n_qubits >= num_qubits)
        try:
            min_dist, min_idx_pair = min_normalized_circ_dist_in_backend_list(bm.get(circ_name), backend_list, **transpile_args)
            print(f"{circ_name:<24}{min_dist:.3f}\t\t{min_idx_pair}")
        except:
            continue

In [7]:
result(bm, provider, seed_transpiler=0)

Benchmark		Min CD (QP)	Min IDX Pair
deutsch_n2              0.116		(2, 3)
dnn_n2                  0.116		(2, 3)
grover_n2               0.116		(2, 3)
iswap_n2                0.116		(2, 3)
quantumwalks_n2         0.117		(2, 3)
basis_change_n3         0.068		(6, 1)
fredkin_n3              0.411		(4, 0)
linearsolver_n3         0.080		(6, 1)
qaoa_n3                 0.570		(6, 0)
teleportation_n3        0.075		(6, 1)
toffoli_n3              0.573		(6, 1)
wstate_n3               0.101		(6, 1)
adder_n4                0.201		(8, 7)
basis_trotter_n4        0.220		(7, 1)
bell_n4                 0.196		(7, 1)
cat_state_n4            0.241		(7, 1)
hs4_n4                  0.327		(7, 6)
inverseqft_n4           0.001		(8, 6)
qft_n4                  0.287		(7, 1)
qrng_n4                 0.001		(8, 6)
variational_n4          0.239		(7, 1)
vqe_n4                  0.194		(7, 1)
vqe_uccsd_n4            0.241		(7, 1)
error_correctiond3_n5   0.220		(7, 1)
lpn_n5                  0.194		(7, 4)
pea_n5        